# Data wrangling
Before we can display our data on Tableau we'll have to prepare it

In [1]:
import pandas as pd
import datetime
import numpy as np

df = pd.read_csv('country_vaccinations.csv')
print(f'The original size of the date: {df.shape}')
df.head()

The original size of the date: (62363, 15)


,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://reliefweb.int/sites/reliefweb.int/file...
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://reliefweb.int/sites/reliefweb.int/file...
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://reliefweb.int/sites/reliefweb.int/file...
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://reliefweb.int/sites/reliefweb.int/file...
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://reliefweb.int/sites/reliefweb.int/file...


Getting rid of the extra rows and changing the one row in the datetime formate and changing missing values to zero. Also dropping countries with no data points for the autumn months

In [18]:
df['date'] = pd.to_datetime(df['date'])
df = df.fillna(0)
drops = ['iso_code', 'people_vaccinated', 'people_fully_vaccinated', 'daily_vaccinations',
        'daily_vaccinations_raw','total_vaccinations','source_website','source_name',
        'daily_vaccinations_per_million','people_vaccinated_per_hundred','total_vaccinations_per_hundred',
        'vaccines']
df = df.drop(drops, axis=1)
df = df[df.country != 'Falkland Islands']
df = df[df.country != 'Nauru']
df = df[df.country != 'Niue']
df = df[df.country != 'Saint Helena']
df = df[df.country != 'Turkmenistan']

creating the new data frame

In [19]:
def growth_finder(country):
    country_df = df[df['country']==country]
    country_df.index = country_df['date']
    country_df = country_df.drop('date', axis=1)
    country_df = country_df.loc[datetime.date(year=2021,month=9,day=1):datetime.date(year=2021,month=11,day=30)]
    growth = country_df.iloc[-1,1]-country_df.iloc[0,1]
    growth_df.loc[len(growth_df)] = [country, growth]

countries = df.country.unique()
growth_df = pd.DataFrame(columns=['country','growth'])

for country in countries:
    growth_finder(country)

growth_df.head()

,country,growth
0,Afghanistan,8.95
1,Albania,10.87
2,Algeria,12.08
3,Andorra,64.04
4,Angola,-2.71


In [20]:
for i,x in enumerate(growth_df['growth']):
    if x<0:
        growth_df=growth_df.drop(index=i)
len(growth_df)

197

In [21]:
growth_df.to_csv("vaccine_growth.csv", index=False)

# Manufacturer dataset

In [48]:
df = pd.read_csv('country_vaccinations_by_manufacturer.csv')
df['date'] = pd.to_datetime(df['date'])
df.index = pd.to_datetime(df['date'])
print(f'The size of the original data {df.shape}')
df.head()

The size of the original data (23859, 4)


,location,date,vaccine,total_vaccinations
date,,,,
2021-01-08,Austria,2021-01-08,Johnson&Johnson,0
2021-01-08,Austria,2021-01-08,Moderna,0
2021-01-08,Austria,2021-01-08,Oxford/AstraZeneca,0
2021-01-08,Austria,2021-01-08,Pfizer/BioNTech,31348
2021-01-15,Austria,2021-01-15,Johnson&Johnson,0


In [49]:
df.location.unique()            # filter the date to keep just the countries we're interested in
df = df[(df['location']=='United States') | (df['location']=='European Union') | (df['location']=='Hong Kong')
       | (df['location']=='Ukraine')]

columns = df.vaccine.unique()
countries = df.location.unique()
idx = df.date.unique()
countries_to_save = []

for i,x in enumerate(countries):        # getting the file names ready to be saved
    if ' ' in x:
        new=x.replace(' ','_')
        countries_to_save.append(new)
    else:
        countries_to_save.append(x)


In [51]:
def country_vaccines(country):                          # function to prepare the data for Tableau
    new_df = df[df['location']==country]
    df_to_save = pd.DataFrame(index=idx, columns=columns).fillna(0)
    for i, row in new_df.iterrows():
        date, vaccine, number = row[1], row[2], row[3]
        df_to_save.loc[date, vaccine] = number
        df_to_save = df_to_save.loc[datetime.date(year=2021,month=9,day=1):datetime.date(year=2021,month=11,day=30)]
        df_to_save = df_to_save.replace(0,np.nan)
        df_to_save = df_to_save.fillna(method='ffill')
    return df_to_save
    
for i in range(len(countries)):
    df_to_save = country_vaccines(countries[i])
    df_to_save.to_csv(countries_to_save[i]+'_vaccines.csv', index=True)

# Vaccines Proportions 

In [16]:
EU = pd.read_csv("European_Union_vaccines.csv", index_col=0)
Hong_Kong = pd.read_csv("Hong_Kong_vaccines.csv", index_col=0)
Ukraine = pd.read_csv("Ukraine_vaccines.csv", index_col=0)
USA = pd.read_csv("United_States_vaccines.csv", index_col=0)
countries = [EU, Hong_Kong, Ukraine, USA]
countries_names=['EU','Hong_Kong','Ukraine','USA']

In [20]:
def proportions(df,i):
    dic={}
    for col in df.columns:
        value = df[col][-1]
        if value>0:
            dic[col]=value
    df_to_save = pd.DataFrame(dic, index=[countries_names[i]])
    df_to_save.to_csv(countries_names[i]+'.csv', index=True)

    
for i in range(len(countries)):
    proportions(countries[i],i)

In [10]:
countries[-1]

,Pfizer/BioNTech,Sinovac,Oxford/AstraZeneca,Johnson&Johnson,Moderna,Sinopharm/Beijing,Sputnik V
2021-09-01,210798052.0,NaN,NaN,14392613.0,145736021.0,NaN,NaN
2021-09-02,211373074.0,NaN,NaN,14420976.0,145967563.0,NaN,NaN
2021-09-03,212319148.0,NaN,NaN,14460971.0,146379359.0,NaN,NaN
2021-09-04,212979842.0,NaN,NaN,14492832.0,146657321.0,NaN,NaN
2021-09-05,210798052.0,NaN,NaN,14392613.0,145736021.0,NaN,NaN
...,...,...,...,...,...,...,...
2021-11-26,210798052.0,NaN,NaN,14392613.0,145736021.0,NaN,NaN
2021-11-27,210798052.0,NaN,NaN,14392613.0,145736021.0,NaN,NaN
2021-11-28,210798052.0,NaN,NaN,14392613.0,145736021.0,NaN,NaN
2021-11-29,266730650.0,NaN,NaN,16597641.0,175430932.0,NaN,NaN
